In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import livenet
importlib.reload(livenet)
core = livenet.core
import simple_log
simple_log.level = simple_log.LogLevel.INFO
from simple_log import LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: <object object at 0x7f519a103940>
2.2.1+cu121


'%.4f'

In [52]:
downscale = (14, 14)
train_x, train_y = livenet.datasets.to_plain(*livenet.datasets.get_mnist_train(), downscale=downscale, to_odd=True, to_gray=True)
n_hidden = 2

# downscale = (8, 8)
# train_x, train_y = livenet.datasets.to_plain(*livenet.datasets.get_cifar10_train(), downscale=downscale, to_odd=False, to_gray=True)
# n_hidden = 12

n_out = np.max(train_y.numpy()) + 1



In [59]:
context = core.livenet.Context()
network = livenet.nets.create_perceptron(train_x.shape[1], n_hidden, n_out, context)
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=50)

LiveNet


In [60]:
network.context.regularization_l1 = 0.01
optimizer.learning_rate = 0.01
trainer.step(5000)


Iˈ0.000 1.018 = 1.000+0.018reg params=16 livenet/net_trainer.py:93
Iˈ0.113 1.009 = 0.989+0.019reg params=16 livenet/net_trainer.py:93
Iˈ0.259 0.952 = 0.910+0.042reg params=16 livenet/net_trainer.py:93
Iˈ0.378 0.874 = 0.802+0.072reg params=15 livenet/net_trainer.py:93
Iˈ0.486 0.829 = 0.733+0.097reg params=15 livenet/net_trainer.py:93
Iˈ0.599 0.806 = 0.693+0.113reg params=14 livenet/net_trainer.py:93
Iˈ0.698 0.811 = 0.683+0.128reg params=13 livenet/net_trainer.py:93
Iˈ0.795 0.832 = 0.683+0.149reg params=12 livenet/net_trainer.py:93
Iˈ0.839 N0 became dangle, total dangle = 1 livenet/core/death.py:55
Iˈ0.839 S0 became useless at tick 423 livenet/core/livenet.py:56
Iˈ0.884 0.873 = 0.708+0.165reg params=10 livenet/net_trainer.py:93
Iˈ0.962 0.871 = 0.713+0.158reg params=10 livenet/net_trainer.py:93
Iˈ1.036 0.830 = 0.694+0.137reg params=10 livenet/net_trainer.py:93
Iˈ1.108 0.801 = 0.683+0.118reg params=10 livenet/net_trainer.py:93
Iˈ1.182 0.806 = 0.704+0.102reg params=10 livenet/net_trainer.py

In [58]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
f"accuracy {len(diff[diff == 0]) / len(diff): .3f}"


'accuracy  0.784'

In [14]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000. + ord(x[0][1]) / 1e6):
    print(name, f"{p.item():.1f}")



D0 -11.3
D1 11.3
N1 9.0
N1->D0 2.9
N1->D1 -0.4
S1->N1 -4.1
S2->N1 5.9
S3->N1 1.2


In [ ]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.grid()
#plt.plot(values)

accum = core.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
accum.plot()


D0 -14.3
D1 14.3
N1 11.9
N1->D0 -0.1
N1->D1 -3.0
S1->N1 -1.2
S2->N1 2.9
S3->N1 1.3


In [52]:
network._parameters


OrderedDict([('D0',
              Parameter containing:
              tensor(24.0669, requires_grad=True)),
             ('D1',
              Parameter containing:
              tensor(-24.0669, requires_grad=True)),
             ('N1',
              Parameter containing:
              tensor(13.0908, requires_grad=True)),
             ('S6->N1',
              Parameter containing:
              tensor(0.7347, requires_grad=True)),
             ('S8->N1',
              Parameter containing:
              tensor(-0.6229, requires_grad=True)),
             ('S9->N1',
              Parameter containing:
              tensor(-1.5580, requires_grad=True)),
             ('S11->N1',
              Parameter containing:
              tensor(-0.5902, requires_grad=True)),
             ('N1->D0',
              Parameter containing:
              tensor(-1.7404, requires_grad=True)),
             ('N1->D1',
              Parameter containing:
              tensor(1.5365, requires_grad=True))])

In [ ]:
core.livenet.export_onnx(network, "/home/spometun/model.onnx")
